In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math

from Clust.clust.ML.regression_JS import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.ML.common.common import p3_training as p3
from Clust.clust.transformation.purpose.machineLearning import LSTMData

# Set Param

In [2]:
param = {
    "d_feat" : 6, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

args = {
    "output_folder_name" : "./ada_rnn_model",
    "save_model_name" : "test_adarnn_model_by_origin_data",
    "lr" : 5e-4,
    "n_epochs" : 200,
    "num_layers" : 2, 
    "len_seq" : 24,
    "pre_epoch" : 40,
    "len_win" : 0,
    "dw" : 0.5,
    "early_stop" :40
}
k = 2

# get Data

In [3]:
# read original data
data_pickle = pd.read_pickle("original_data.pkl")
original_data = data_pickle["Dongsi"]

In [4]:
len(original_data), original_data[0].shape, original_data[2].shape

(3, (1460, 24, 6), (1460,))

In [5]:
# original data 에서 data_x, data_y 나누기
data_x = original_data[0]
data_y = original_data[2]

In [6]:
# set time
original_data_start_time = datetime.datetime(2013, 3, 1, 0, 0)

## train data time
train_start_time = '2013-03-01 00:00:00'
train_end_time = '2016-06-30 23:00:00'

## valid data time
valid_start_time='2016-7-2 00:00:00'
valid_end_time='2016-10-30 23:00:00'

In [7]:
# split train & validation data -> get train_x, train_y, valid_x, valid_y
index_start=(pd.to_datetime(train_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(train_end_time) - original_data_start_time).days

train_x=data_x[index_start: index_end + 1]
train_y=data_y[index_start: index_end + 1]

index_start=(pd.to_datetime(valid_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(valid_end_time) - original_data_start_time).days

valid_x=data_x[index_start: index_end + 1]
valid_y=data_y[index_start: index_end + 1]

In [8]:
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

((1218, 24, 6), (1218,), (121, 24, 6), (121,))

# Data Loader

In [9]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'d_feat': 6, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [10]:
train_loader_list, valid_loader = adarnn.create_trainloader(train_x, train_y, valid_x, valid_y, k, train_start_time, train_end_time)

# Training

In [11]:
## test loader 제거 작업 시작
adarnn.main_transfer(train_loader_list, valid_loader, args)

[2023-03-10 16:45:21] - create model...
[2023-03-10 16:45:21] - Epoch: 0
[2023-03-10 16:45:21] - training...


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 13.79it/s]

[2023-03-10 16:45:21] - evaluating...



Valid: 100%|█████████████████████████████████████| 2/2 [00:00<00:00, 696.90it/s]

[2023-03-10 16:45:21] - train 0.056806, valid 0.135879
[2023-03-10 16:45:21] - Epoch: 1
[2023-03-10 16:45:21] - training...


...

[2023-03-10 16:45:44] - train 0.022643, valid 0.276574
[2023-03-10 16:45:44] - Epoch: 54
[2023-03-10 16:45:44] - training...



100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 24.61it/s]

[2023-03-10 16:45:44] - evaluating...



Valid: 100%|█████████████████████████████████████| 2/2 [00:00<00:00, 424.40it/s]

[2023-03-10 16:45:44] - train 0.022345, valid 0.919755
[2023-03-10 16:45:44] - early stop
[2023-03-10 16:45:44] - best val score: 0.00040685906424187124 @ 14
[2023-03-10 16:45:44] - Finished.
